# ICBS: Supply, Cost, and Price Surfaces

Exploring the 3D geometry of ICBS supply curves, cost function, and price surfaces.

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact, FloatSlider, IntSlider, Layout
import ipywidgets as widgets

## Core ICBS Math

In [ ]:
# Core ICBS functions
def cost_function(s_L, s_S, lambda_val):
    """C = λ√(sL² + sS²) - the cost cone"""
    return lambda_val * np.sqrt(s_L**2 + s_S**2)

def price_long(s_L, s_S, lambda_val):
    """Price of LONG = ∂C/∂sL = λ · sL / √(sL² + sS²)"""
    norm = np.sqrt(s_L**2 + s_S**2)
    return lambda_val * s_L / norm if norm > 0 else 0

def price_short(s_L, s_S, lambda_val):
    """Price of SHORT = ∂C/∂sS = λ · sS / √(sL² + sS²)"""
    norm = np.sqrt(s_L**2 + s_S**2)
    return lambda_val * s_S / norm if norm > 0 else 0

In [ ]:
def visualize_supply_and_cost(max_supply=100, lambda_val=1.0, resolution=50):
    """Visualize the supply space and cost cone in 3D."""
    # Create supply grid
    s_grid = np.linspace(0, max_supply, resolution)
    S_L, S_S = np.meshgrid(s_grid, s_grid)
    
    # Calculate cost surface
    C = cost_function(S_L, S_S, lambda_val)
    
    # Create figure
    fig = go.Figure()
    
    # Add cost cone surface
    fig.add_trace(
        go.Surface(
            x=S_L, y=S_S, z=C,
            colorscale='Viridis',
            opacity=0.85,
            name='Cost Cone',
            colorbar=dict(
                title='Cost ($)',
                x=1.0
            ),
            hovertemplate='<b>Supply & Cost</b><br>' +
                          'LONG: %{x:.1f}<br>' +
                          'SHORT: %{y:.1f}<br>' +
                          'Cost: $%{z:.2f}<br>' +
                          '<extra></extra>'
        )
    )
    
    # Add grid lines on base for clarity
    for val in np.linspace(0, max_supply, 5):
        fig.add_trace(
            go.Scatter3d(
                x=[val, val],
                y=[0, max_supply],
                z=[0, 0],
                mode='lines',
                line=dict(color='gray', width=1, dash='dash'),
                showlegend=False,
                hoverinfo='skip'
            )
        )
        fig.add_trace(
            go.Scatter3d(
                x=[0, max_supply],
                y=[val, val],
                z=[0, 0],
                mode='lines',
                line=dict(color='gray', width=1, dash='dash'),
                showlegend=False,
                hoverinfo='skip'
            )
        )
    
    # Update layout
    fig.update_layout(
        title=dict(
            text=f'<b>ICBS Cost Cone</b><br><sub>C = λ√(s<sub>L</sub>² + s<sub>S</sub>²) with λ={lambda_val}</sub>',
            x=0.5,
            xanchor='center'
        ),
        scene=dict(
            xaxis_title='LONG Supply (sL)',
            yaxis_title='SHORT Supply (sS)',
            zaxis_title='Total Cost C',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.3)
            ),
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=0.8)
        ),
        width=900,
        height=700,
        margin=dict(l=0, r=0, t=80, b=0)
    )
    
    fig.show()
    
    print("\n" + "="*60)
    print("COST CONE PROPERTIES")
    print("="*60)
    print(f"• λ (lambda) = {lambda_val}")
    print(f"• At (50, 50): Cost = ${cost_function(50, 50, lambda_val):.2f}")
    print(f"• At (100, 0): Cost = ${cost_function(100, 0, lambda_val):.2f}")
    print(f"• At (0, 100): Cost = ${cost_function(0, 100, lambda_val):.2f}")
    print(f"• Geometry: Circular cone centered at origin")
    print(f"• Key insight: Cost increases with TOTAL liquidity (Euclidean norm)")

In [ ]:
# Interactive supply and cost visualization
interact(
    visualize_supply_and_cost,
    max_supply=IntSlider(
        min=50, max=200, step=10, value=100,
        description='Max Supply:',
        style={'description_width': '100px'},
        layout=Layout(width='500px')
    ),
    lambda_val=FloatSlider(
        min=0.5, max=2.0, step=0.1, value=1.0,
        description='Lambda (λ):',
        style={'description_width': '100px'},
        layout=Layout(width='500px')
    ),
    resolution=IntSlider(
        min=20, max=100, step=10, value=50,
        description='Resolution:',
        style={'description_width': '100px'},
        layout=Layout(width='500px')
    )
);

print("\n🔧 CONTROLS:")
print("  • Max Supply: Scale of the supply axes")
print("  • Lambda (λ): Cost multiplier (slope of the cone)")
print("  • Resolution: Surface detail (higher = smoother but slower)")

interactive(children=(IntSlider(value=100, description='Max Supply:', layout=Layout(width='500px'), max=200, m…


🔧 CONTROLS:
  • Max Supply: Scale of the supply axes
  • Lambda (λ): Cost multiplier (slope of the cone)
  • Resolution: Surface detail (higher = smoother but slower)


## Price Surfaces

The prices are the partial derivatives of the cost function: ∂C/∂sL and ∂C/∂sS.

In [ ]:
def visualize_prices(max_supply=100, lambda_val=1.0, resolution=50):
    """Visualize LONG and SHORT price surfaces in 3D."""
    # Create supply grid (avoid zero to prevent division issues)
    s_grid = np.linspace(0.1, max_supply, resolution)
    S_L, S_S = np.meshgrid(s_grid, s_grid)
    
    # Calculate price surfaces
    P_L = price_long(S_L, S_S, lambda_val)
    P_S = price_short(S_L, S_S, lambda_val)
    
    # Create subplots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('LONG Price Surface', 'SHORT Price Surface'),
        specs=[[{'type': 'surface'}, {'type': 'surface'}]],
        horizontal_spacing=0.05
    )
    
    # Add LONG price surface
    fig.add_trace(
        go.Surface(
            x=S_L, y=S_S, z=P_L,
            colorscale='Greens',
            opacity=0.85,
            name='p_L',
            colorbar=dict(
                title='p_L ($)',
                x=0.43,
                len=0.8
            ),
            hovertemplate='<b>LONG Price</b><br>' +
                          'sL: %{x:.1f}<br>' +
                          'sS: %{y:.1f}<br>' +
                          'p_L: $%{z:.3f}<br>' +
                          '<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Add SHORT price surface
    fig.add_trace(
        go.Surface(
            x=S_L, y=S_S, z=P_S,
            colorscale='Reds',
            opacity=0.85,
            name='p_S',
            colorbar=dict(
                title='p_S ($)',
                x=1.0,
                len=0.8
            ),
            hovertemplate='<b>SHORT Price</b><br>' +
                          'sL: %{x:.1f}<br>' +
                          'sS: %{y:.1f}<br>' +
                          'p_S: $%{z:.3f}<br>' +
                          '<extra></extra>'
        ),
        row=1, col=2
    )
    
    # Update layout
    fig.update_layout(
        title=dict(
            text=f'<b>ICBS Price Surfaces</b><br><sub>p_L = λs_L/√(s_L²+s_S²) and p_S = λs_S/√(s_L²+s_S²) with λ={lambda_val}</sub>',
            x=0.5,
            xanchor='center'
        ),
        scene=dict(
            xaxis_title='LONG Supply (sL)',
            yaxis_title='SHORT Supply (sS)',
            zaxis_title='Price p_L',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.3)),
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=0.6)
        ),
        scene2=dict(
            xaxis_title='LONG Supply (sL)',
            yaxis_title='SHORT Supply (sS)',
            zaxis_title='Price p_S',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.3)),
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=0.6)
        ),
        width=1400,
        height=600,
        margin=dict(l=0, r=0, t=100, b=0)
    )
    
    fig.show()
    
    print("\n" + "="*60)
    print("PRICE SURFACE PROPERTIES")
    print("="*60)
    print(f"• λ (lambda) = {lambda_val}")
    print(f"\nSample prices at key points:")
    for sL, sS in [(50, 50), (80, 20), (20, 80), (90, 10)]:
        pL = price_long(sL, sS, lambda_val)
        pS = price_short(sL, sS, lambda_val)
        print(f"  At ({sL:3d}, {sS:3d}): p_L=${pL:.3f}, p_S=${pS:.3f}, sum=${pL+pS:.3f}")
    
    print(f"\n• Key insight: p_L + p_S = λ (always)")
    print(f"• LONG price high when sL >> sS (high LONG supply)")
    print(f"• SHORT price high when sS >> sL (high SHORT supply)")
    print(f"• Balanced at diagonal (sL = sS): both prices equal λ/√2")

In [ ]:
# Interactive price surface visualization
interact(
    visualize_prices,
    max_supply=IntSlider(
        min=50, max=200, step=10, value=100,
        description='Max Supply:',
        style={'description_width': '100px'},
        layout=Layout(width='500px')
    ),
    lambda_val=FloatSlider(
        min=0.5, max=2.0, step=0.1, value=1.0,
        description='Lambda (λ):',
        style={'description_width': '100px'},
        layout=Layout(width='500px')
    ),
    resolution=IntSlider(
        min=20, max=100, step=10, value=50,
        description='Resolution:',
        style={'description_width': '100px'},
        layout=Layout(width='500px')
    )
);

print("\n🔧 CONTROLS:")
print("  • Max Supply: Scale of the supply axes")
print("  • Lambda (λ): Maximum price (when one side → 100%)")
print("  • Resolution: Surface detail (higher = smoother but slower)")

## Path Dependence: Order Matters for Cost

The cost curve is **fixed and deterministic** - it exists independent of trading.  
But **trading paths matter** - different sequences reaching the same endpoint cost different amounts.

def calculate_path_cost(path_points, lambda_val):
    """Calculate total cost along a path on the cone."""
    total_cost = 0
    for i in range(len(path_points) - 1):
        sL_start, sS_start = path_points[i]
        sL_end, sS_end = path_points[i + 1]
        cost_start = cost_function(sL_start, sS_start, lambda_val)
        cost_end = cost_function(sL_end, sS_end, lambda_val)
        total_cost += (cost_end - cost_start)
    return total_cost

def visualize_path_dependence(target_sL=80, target_sS=80, lambda_val=1.0):
    """Show how different trading paths to the same endpoint cost different amounts."""
    start = (50, 50)  # Always start at 50/50
    end = (target_sL, target_sS)
    
    # Define three different paths to the same endpoint
    paths = {
        'Direct': [start, end],
        'LONG first': [start, (target_sL, 50), end],
        'SHORT first': [start, (50, target_sS), end],
    }
    
    colors = {'Direct': 'blue', 'LONG first': 'green', 'SHORT first': 'red'}
    
    # Calculate costs
    costs = {name: calculate_path_cost(path, lambda_val) for name, path in paths.items()}
    
    # Create 3D visualization
    max_val = max(target_sL, target_sS, 100)
    s_grid = np.linspace(0, max_val, 40)
    S_L, S_S = np.meshgrid(s_grid, s_grid)
    C = cost_function(S_L, S_S, lambda_val)
    
    fig = go.Figure()
    
    # Add cost cone (transparent)
    fig.add_trace(
        go.Surface(
            x=S_L, y=S_S, z=C,
            colorscale='Greys',
            opacity=0.2,
            showscale=False,
            hoverinfo='skip'
        )
    )
    
    # Add each path
    for name, path in paths.items():
        sL_vals = [p[0] for p in path]
        sS_vals = [p[1] for p in path]
        cost_vals = [cost_function(p[0], p[1], lambda_val) for p in path]
        
        fig.add_trace(
            go.Scatter3d(
                x=sL_vals,
                y=sS_vals,
                z=cost_vals,
                mode='lines+markers',
                name=f'{name}: ${costs[name]:.2f}',
                line=dict(color=colors[name], width=8),
                marker=dict(size=10, color=colors[name]),
                text=[f'Step {i}' for i in range(len(path))],
                hovertemplate='<b>%{fullData.name}</b><br>' +
                              'sL: %{x:.1f}<br>' +
                              'sS: %{y:.1f}<br>' +
                              'Height: $%{z:.2f}<br>' +
                              '<extra></extra>'
            )
        )
    
    # Mark start and end
    start_cost = cost_function(start[0], start[1], lambda_val)
    end_cost = cost_function(end[0], end[1], lambda_val)
    
    fig.add_trace(
        go.Scatter3d(
            x=[start[0]],
            y=[start[1]],
            z=[start_cost],
            mode='markers',
            name='START',
            marker=dict(size=15, color='black', symbol='diamond'),
            showlegend=True
        )
    )
    
    fig.add_trace(
        go.Scatter3d(
            x=[end[0]],
            y=[end[1]],
            z=[end_cost],
            mode='markers',
            name='END',
            marker=dict(size=15, color='gold', symbol='diamond'),
            showlegend=True
        )
    )
    
    # Layout
    fig.update_layout(
        title=dict(
            text=f'<b>Path Dependence on Cost Cone</b><br><sub>Same endpoint, different costs (λ={lambda_val})</sub>',
            x=0.5,
            xanchor='center'
        ),
        scene=dict(
            xaxis_title='LONG Supply (sL)',
            yaxis_title='SHORT Supply (sS)',
            zaxis_title='Cost C',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.3)),
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=0.8)
        ),
        width=1000,
        height=700,
        margin=dict(l=0, r=0, t=80, b=0),
        legend=dict(x=0.7, y=0.95)
    )
    
    fig.show()
    
    # Analysis
    print("\n" + "="*70)
    print("PATH COST ANALYSIS")
    print("="*70)
    print(f"Start: ({start[0]}, {start[1]}) → Height: ${start_cost:.2f}")
    print(f"End:   ({end[0]}, {end[1]}) → Height: ${end_cost:.2f}")
    print(f"Height gain: ${end_cost - start_cost:.2f}")
    print()
    
    for name in ['Direct', 'LONG first', 'SHORT first']:
        cost = costs[name]
        diff = cost - costs['Direct']
        pct = (diff / costs['Direct'] * 100) if costs['Direct'] > 0 else 0
        print(f"{name:15s}: ${cost:6.2f}  (vs Direct {diff:+.2f} or {pct:+.1f}%)")
    
    print("\n" + "="*70)
    print("KEY INSIGHTS")
    print("="*70)
    print("1. The cost cone is FIXED - it doesn't change with trading order")
    print("2. BUT different paths cost different amounts to traverse")
    print("3. Direct diagonal paths are usually most efficient")
    print("4. Detours increase total cost due to the cone's curvature")
    print("5. This is why MEV and trade ordering matters in ICBS!")

In [ ]:
# Interactive path dependence visualization
interact(
    visualize_path_dependence,
    target_sL=IntSlider(
        min=60, max=150, step=10, value=80,
        description='Target sL:',
        style={'description_width': '100px'},
        layout=Layout(width='500px')
    ),
    target_sS=IntSlider(
        min=60, max=150, step=10, value=80,
        description='Target sS:',
        style={'description_width': '100px'},
        layout=Layout(width='500px')
    ),
    lambda_val=FloatSlider(
        min=0.5, max=2.0, step=0.1, value=1.0,
        description='Lambda (λ):',
        style={'description_width': '100px'},
        layout=Layout(width='500px')
    )
);

print("\n🔧 CONTROLS:")
print("  • Target sL/sS: Where all paths end up")
print("  • Lambda (λ): Cone steepness")
print("\n💡 Try different endpoints to see how path costs vary!")

In [ ]:
def calculate_path_cost(path_points, lambda_val):
    """Calculate total cost along a path on the cone."""
    total_cost = 0
    for i in range(len(path_points) - 1):
        sL_start, sS_start = path_points[i]
        sL_end, sS_end = path_points[i + 1]
        cost_start = cost_function(sL_start, sS_start, lambda_val)
        cost_end = cost_function(sL_end, sS_end, lambda_val)
        total_cost += (cost_end - cost_start)
    return total_cost

def visualize_path_dependence(target_sL=80, target_sS=80, lambda_val=1.0):
    """Show how different trading paths to the same endpoint cost different amounts."""
    start = (50, 50)  # Always start at 50/50
    end = (target_sL, target_sS)
    
    # Define three different paths to the same endpoint
    paths = {
        'Direct': [start, end],
        'LONG first': [start, (target_sL, 50), end],
        'SHORT first': [start, (50, target_sS), end],
    }
    
    colors = {'Direct': 'blue', 'LONG first': 'green', 'SHORT first': 'red'}
    
    # Calculate costs
    costs = {name: calculate_path_cost(path, lambda_val) for name, path in paths.items()}
    
    # Create 3D visualization
    max_val = max(target_sL, target_sS, 100)
    s_grid = np.linspace(0, max_val, 40)
    S_L, S_S = np.meshgrid(s_grid, s_grid)
    C = cost_function(S_L, S_S, lambda_val)
    
    fig = go.Figure()
    
    # Add cost cone (transparent)
    fig.add_trace(
        go.Surface(
            x=S_L, y=S_S, z=C,
            colorscale='Greys',
            opacity=0.2,
            showscale=False,
            hoverinfo='skip'
        )
    )
    
    # Add each path
    for name, path in paths.items():
        sL_vals = [p[0] for p in path]
        sS_vals = [p[1] for p in path]
        cost_vals = [cost_function(p[0], p[1], lambda_val) for p in path]
        
        fig.add_trace(
            go.Scatter3d(
                x=sL_vals,
                y=sS_vals,
                z=cost_vals,
                mode='lines+markers',
                name=f'{name}<br>Cost: ${costs[name]:.2f}',
                line=dict(color=colors[name], width=8),
                marker=dict(size=10, color=colors[name]),
                text=[f'Step {i}' for i in range(len(path))],
                hovertemplate='<b>%{fullData.name}</b><br>' +
                              'sL: %{x:.1f}<br>' +
                              'sS: %{y:.1f}<br>' +
                              'Height: $%{z:.2f}<br>' +
                              '<extra></extra>'
            )
        )
    
    # Mark start and end
    start_cost = cost_function(start[0], start[1], lambda_val)
    end_cost = cost_function(end[0], end[1], lambda_val)
    
    fig.add_trace(
        go.Scatter3d(
            x=[start[0]],
            y=[start[1]],
            z=[start_cost],
            mode='markers',
            name='START',
            marker=dict(size=15, color='black', symbol='diamond'),
            showlegend=True
        )
    )
    
    fig.add_trace(
        go.Scatter3d(
            x=[end[0]],
            y=[end[1]],
            z=[end_cost],
            mode='markers',
            name='END',
            marker=dict(size=15, color='gold', symbol='diamond'),
            showlegend=True
        )
    )
    
    # Layout
    fig.update_layout(
        title=dict(
            text=f'<b>Path Dependence on the Cost Cone</b><br><sub>Same endpoint, different costs (λ={lambda_val})</sub>',
            x=0.5,
            xanchor='center'
        ),
        scene=dict(
            xaxis_title='LONG Supply (sL)',
            yaxis_title='SHORT Supply (sS)',
            zaxis_title='Cost C',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.3)),
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=0.8)
        ),
        width=1000,
        height=700,
        margin=dict(l=0, r=0, t=80, b=0),
        legend=dict(x=0.7, y=0.95)
    )
    
    fig.show()
    
    # Analysis
    print("\n" + "="*70)
    print("PATH COST ANALYSIS")
    print("="*70)
    print(f"Start: ({start[0]}, {start[1]}) → Height: ${start_cost:.2f}")
    print(f"End:   ({end[0]}, {end[1]}) → Height: ${end_cost:.2f}")
    print(f"Height gain: ${end_cost - start_cost:.2f}")
    print()
    
    for name in ['Direct', 'LONG first', 'SHORT first']:
        cost = costs[name]
        diff = cost - costs['Direct']
        pct = (diff / costs['Direct'] * 100) if costs['Direct'] > 0 else 0
        print(f"{name:15s}: ${cost:6.2f}  (Direct {diff:+.2f} or {pct:+.1f}%)")
    
    print("\n" + "="*70)
    print("KEY INSIGHTS")
    print("="*70)
    print("1. The cost cone is FIXED - it doesn't change with trading order")
    print("2. BUT different paths cost different amounts to traverse")
    print("3. Direct diagonal paths are usually most efficient")
    print("4. Detours increase total cost due to the cone's curvature")
    print("5. This is why MEV and trade ordering matters in ICBS!")